Using Bayesian inference for learning synthesis-structure-property relationship via coregionalized piecewise function determination.
Examples for paper:
Code and examples are presented for 1D algorithms.

Table of Content:
* Libraries to Install
* Import Libraries
* 1D Case:
    * .py file for 1D functions.
    * 1D Edge Case Challenges
         * Set up challenge data
         * Run algorithms (1 core scripts) and collect data
         * Visualize results
         * Compute performance measures available in paper Table 1.

### Libraries to Install: ** please see requirements.txt and SAGEn_241025a.yml **

### Import Libraries

In [ ]:
from sage_1D_functions_230804a import *

import matplotlib.pyplot as plt

import numpyro
import numpy as np
from numpy.random import default_rng

import torch

import dill
from torch.distributions import constraints

from torch.nn import Parameter
from torch.nn import Softmax
from torch.nn.functional import one_hot

import pyro
from pyro.infer import MCMC, NUTS, HMC, Predictive, SVI, Trace_ELBO
import pyro.contrib.gp as gp
import pyro.distributions as dist
from pyro.infer.autoguide import initialization as init

from scipy.spatial.distance import pdist, squareform
from scipy.spatial import Voronoi
from scipy.stats import multivariate_normal, entropy
import scipy.io as sio
from scipy.special import softmax as softnp
from scipy.stats.mstats import mquantiles
from scipy.interpolate import griddata
from scipy.stats import gamma, gennorm

import jax
import jax.numpy as jnp
from jax import config
config.update("jax_enable_x64", True)
from jax.lax import dynamic_slice
from jax.nn import one_hot as jax_one_hot

from numpyro.infer import MCMC as nMCMC
from numpyro.infer import NUTS as nNUTS
from numpyro.infer import Predictive as nPredictive
import numpyro.distributions as ndist

import tensorflow_probability as tfp

from sklearn.metrics import precision_recall_fscore_support as prfs
from sklearn.metrics.cluster import fowlkes_mallows_score as fmi
from sklearn.metrics import fowlkes_mallows_score as fms

torch.set_default_dtype(torch.float64)
from tqdm import trange

from applied_active_learning_191228a import *
from cameo_240821a import *

## 1-dimensional Cases

### 1D Functions: Create .py file

In [ ]:
%%writefile sage_1D_functions_230804a.py

import matplotlib.pyplot as plt
import numpy as np
from numpy.random import default_rng

import torch
from torch.distributions import constraints
from torch.nn.functional import one_hot

import pyro
from pyro.infer import MCMC, NUTS, HMC, Predictive, SVI, Trace_ELBO
import pyro.distributions as dist

import gpflow
from scipy.stats import entropy
import tensorflow_probability as tfp

torch.set_default_dtype(torch.float64)
from tqdm import trange


# Data Generation ---------------------------
def simple_challenge_1D_2regions(X):
    L = 10.
    yg1 = .5*np.exp(-.5*(X-3.5/L)**2 / (1./L))
    yg1[X > 7./L] = 0
    yg2 = 2*np.exp(-.5*(X-7./L)**2 / (.25/L))
    yg2[X < 7./L] = 0
    y = yg1 + yg2
    return y

# -------- BI for Models --------------------------
def SAGE_1D_230628a(xs, ys, xf, yf, Xpred, num_regions = 2, num_samples = 100, num_warmup = 10, 
                                      gpr_var_bounds = torch.tensor([.01, 2.]), gpr_ls_bounds = torch.tensor([.1,1.]),
                                        gpr_noise_bounds = torch.tensor([0.001,.01]), cp_bounds = torch.tensor([0.5, 1.])):
    xs = to_torch(xs).double()
    ys = to_torch(ys).long()
    xf = to_torch(xf).double()
    yf = to_torch(yf).double()
    Xnew_ = to_torch(Xpred).double()

    nuts = MCMC(NUTS(model_SAGE_1D_230628a, target_accept_prob=0.8, max_tree_depth=5),
                num_samples=num_samples, warmup_steps=num_warmup)
    nuts.run(xs, ys, xf, yf, num_regions, gpr_var_bounds = gpr_var_bounds, gpr_ls_bounds = gpr_ls_bounds,
             gpr_noise_bounds = gpr_noise_bounds, cp_bounds = cp_bounds)

    nuts_posterior_samples = nuts.get_samples()

    num_regions = 2
    predictive = Predictive(model_SAGE_1D_230628a, nuts_posterior_samples)(xs, ys, xf, yf, num_regions)

    idx = torch.argmax(predictive['llk'])
    max_llk_sample_cp = nuts_posterior_samples['changepoint'][idx]
    gpc_probs_mllk, gpr_mean_noiseless_mllk, gpr_samples, _, gpr_var_noiseless_mllk = predict_SAGE_1D_230628a(nuts_posterior_samples, idx, xs, Xnew_, num_regions, xf, yf)

    
    
    preds = [predict_SAGE_1D_230628a(nuts_posterior_samples, i, xs, Xnew_, num_regions, xf, yf)
                    for i in trange(nuts_posterior_samples['gpr_var'].shape[0])];
    # gpc_new_probs, gpr_new_mean_mixture, gpr_new_var_mixture
    gpc_new_mean = np.stack([item[0] for item in preds]).mean(axis=0)
    gpc_new_std = np.stack([item[0] for item in preds]).std(axis=0)

    # squeeze assumes 1 functional property:
    gpr_samples_noiseless = np.stack([item[1] for item in preds], axis = 2).squeeze(axis = 1)
    gpr_samples = np.stack([item[2] for item in preds], axis = 2).squeeze(axis = 1)
    gpr_regions = np.stack([item[3] for item in preds], axis = 2)
    
    # assume Mf == 1
    gpr_new_mean = gpr_samples.mean(axis = 1)
    gpr_new_var = gpr_samples.std(axis = 1)**2
    gpr_new_std = np.sqrt(gpr_new_var)
    
    labels_new_est = gpc_new_mean #np.mean(gpc_new_mean,axis=1)
    labels_new_entropy = entropy(gpc_new_mean, axis =1)
        
    cp_samples = np.array(nuts_posterior_samples['changepoint'].flatten())

    # gpc_new_mean, gpc_new_std, gpr_new_mean, gpr_new_var, cp_samples, gpr_mean_noiseless_mllk, gpr_var_noiseless_mllk
    output = {'gpc_mean':gpc_new_mean, 'gpc_std':gpc_new_std,
              'gpr_mean':gpr_new_mean, 'gpr_var':gpr_new_var, 'gpr_std':gpr_new_std,
              'gpr_samples':gpr_samples, 'gpr_samples_noiseless':gpr_samples_noiseless,
              'cp_samples':cp_samples,
              'samples':nuts_posterior_samples,
              'maxllk_gpr_fmean':gpr_mean_noiseless_mllk.squeeze(), 'maxllk_gpr_fvar':gpr_var_noiseless_mllk.squeeze(), 'maxllk_cp':max_llk_sample_cp}
    return output

def SAGE_1D_FP_230628a(xf, yf, Xpred, num_regions = 2, num_samples = 100, num_warmup = 10, 
                                      gpr_var_bounds = torch.tensor([.01, 2.]), gpr_ls_bounds = torch.tensor([.1,1.]),
                                        gpr_noise_bounds = torch.tensor([0.001,.01]), cp_bounds = torch.tensor([0.5, 1.])):
    xf = to_torch(xf).double()
    yf = to_torch(yf).double()
    Xnew_ = to_torch(Xpred).double()

    nuts = MCMC(NUTS(model_SAGE_1D_FP_230628a, target_accept_prob=0.8, max_tree_depth=5),
                num_samples=num_samples, warmup_steps=num_warmup)
    nuts.run(xf, yf, num_regions, gpr_var_bounds = gpr_var_bounds, gpr_ls_bounds = gpr_ls_bounds,
             gpr_noise_bounds = gpr_noise_bounds, cp_bounds = cp_bounds)

    samples = nuts.get_samples()

    num_regions = 2
    predictive = Predictive(model_SAGE_1D_FP_230628a, samples)(xf, yf, num_regions)

    idx = torch.argmax(predictive['llk'])
    max_llk_sample_cp = samples['changepoint'][idx]
    gpc_probs_mllk, gpr_mean_noiseless_mllk, gpr_samples, _, gpr_var_noiseless_mllk = predict_SAGE_1D_FP_230628a(samples, idx, Xnew_, num_regions, xf, yf)
    
    preds = [predict_SAGE_1D_FP_230628a(samples, i, Xnew_, num_regions, xf, yf)
                    for i in trange(samples['gpr_var'].shape[0])];
    # gpc_new_probs, gpr_new_mean_mixture, gpr_new_var_mixture
    gpc_new_mean = np.stack([item[0] for item in preds]).mean(axis=0)
    gpc_new_std = np.stack([item[0] for item in preds]).std(axis=0)

    # squeeze assumes 1 functional property:
    gpr_samples_noiseless = np.stack([item[1] for item in preds], axis = 2).squeeze(axis = 1)
    gpr_samples = np.stack([item[2] for item in preds], axis = 2).squeeze(axis = 1)
    gpr_regions = np.stack([item[3] for item in preds], axis = 2)
    
    # assume Mf == 1
    gpr_new_mean = gpr_samples.mean(axis = 1)
    gpr_new_var = gpr_samples.std(axis = 1)**2
    
    labels_new_est = gpc_new_mean #np.mean(gpc_new_mean,axis=1)
    labels_new_entropy = entropy(gpc_new_mean, axis =1)
    
    cp_samples = np.array(samples['changepoint'].flatten())
    
    # gpc_new_mean, gpc_new_std, gpr_new_mean, gpr_new_var, cp_samples, gpr_mean_noiseless_mllk, gpr_var_noiseless_mllk
    output = {'gpc_mean':gpc_new_mean, 'gpc_std':gpc_new_std,
          'gpr_mean':gpr_new_mean, 'gpr_var':gpr_new_var, 'gpr_samples':gpr_samples, 'gpr_samples_noiseless':gpr_samples_noiseless,
          'cp_samples':cp_samples,
          'maxllk_gpr_fmean':gpr_mean_noiseless_mllk, 'maxllk_gpr_fvar':gpr_var_noiseless_mllk, 'maxllk_cp':max_llk_sample_cp}
    return output

def SAGE_1D_PM_230628a(Xpred, xs, ys, num_regions, numsteps):
    # Can use samples of change_point to define uncertainty.
    # can bin changepoint to make this faster and iterate over bins.
    data = [xs, ys, Xpred, num_regions]

    ker = NUTS(model_SAGE_1D_PM_230628a, jit_compile=True, ignore_jit_warnings=True, max_tree_depth=3)
    posterior = MCMC(ker, num_samples=numsteps, warmup_steps=100)
    posterior.run(data);
    keep_trying = False
    
    nuts_posterior_samples = posterior.get_samples()

    num_regions = 2
    predictive = Predictive(model_SAGE_1D_PM_230628a, nuts_posterior_samples)(data)

    idx = torch.argmax(predictive['llk_pm'])
    max_llk_sample_cp = nuts_posterior_samples['cp'][idx]
    

    s = {k: v.detach().cpu().numpy() for k, v in posterior.get_samples().items()}
    cp_ = np.sort(s['cp'],axis=1).squeeze() # used to sort the change points by location.
    if len(cp_.shape) == 1:
        cp_ = cp_[:,None]
    cp = np.mean(cp_, axis=0)

    Pmu = cp_samples_to_class_prob(cp_, Xpred)
    Pvar = []
    output = {'cat_dist_mean':Pmu, 'cp_samples':cp_, 'maxllk_cp':max_llk_sample_cp}
    return output, posterior

# ------ Models ---------------------
def model_SAGE_1D_FP_230628a(xf, yf, num_regions, gpr_var_bounds = torch.tensor([0.01,.2]), 
                              gpr_ls_bounds = torch.tensor([.1,1.]), gpr_noise_bounds = torch.tensor([0.001, .01]), \
                              cp_bounds = torch.tensor([0., 1.]), gpr_bias_bounds = torch.tensor([-2., 2.])):
    # print('starting')
    noise=torch.tensor(1E-5)
    jitter=torch.tensor(1E-5)
    
    Mf = yf.shape[1]
    Nf = xf.shape[0]

    changepoint_min_bound = cp_bounds[0]*torch.ones((num_regions-1))
    changepoint_max_bound = cp_bounds[1]*torch.ones((num_regions-1))
    gpr_var_min_bound = gpr_var_bounds[0]*torch.ones((num_regions,Mf))
    gpr_var_max_bound = gpr_var_bounds[1]*torch.ones((num_regions,Mf))
    gpr_ls_min_bound = gpr_ls_bounds[0]*torch.ones((num_regions,Mf))
    gpr_ls_max_bound = gpr_ls_bounds[1]*torch.ones((num_regions,Mf))
    gpr_bias_min_bound = gpr_bias_bounds[0]*torch.ones((num_regions,Mf))
    gpr_bias_max_bound = gpr_bias_bounds[1]*torch.ones((num_regions,Mf))
    
    changepoint = pyro.sample('changepoint', dist.Uniform(changepoint_min_bound,changepoint_max_bound ) )
    gpr_noise = pyro.sample("gpr_noise", dist.Uniform(gpr_noise_bounds[0], gpr_noise_bounds[1]))
    gpr_var = pyro.sample("gpr_var", dist.Uniform(gpr_var_min_bound, gpr_var_max_bound))
    gpr_lengthscale = pyro.sample("gpr_lengthscale", dist.Uniform(gpr_ls_min_bound, gpr_ls_max_bound))
    gpr_bias = pyro.sample("gpr_bias", dist.Uniform(gpr_bias_min_bound, gpr_bias_max_bound))
    
    region_labels = change_points_to_labels_torch(changepoint, xf)
    
    probs_fp = one_hot(region_labels, num_regions)
    
    F = torch.zeros((Nf,num_regions,Mf))
    for j in range(Mf):
        for i in range(num_regions):
            with pyro.plate('latent_response' + str(i), Nf):
                eta = pyro.sample('sample'+str(i)+'_Mf'+str(j), dist.Normal(0, 1))
    
            f = compute_f_torch(gpr_var[i,j], gpr_lengthscale[i,j], gpr_bias[i,j], eta, xf)
            F[:,i,j] = f
    
    f_piecewise = torch.zeros((Nf, Mf))
    for j in range(Mf):
        for i in range(num_regions):
            f_piecewise[:,j] = f_piecewise[:,j] + probs_fp[:,i] * F[:,i,j]

    llk = 0.
    for j in range(Mf):
        llk = llk + dist.Normal(f_piecewise[:,j], torch.sqrt( gpr_noise ) ).log_prob(yf[:,j]).sum()
    
    pyro.deterministic("llk", llk)
    pyro.factor("obs", llk)

def model_SAGE_1D_230628a(xs, ys, xf, yf, num_regions, gpr_var_bounds = torch.tensor([0.01,.2]), 
                              gpr_ls_bounds = torch.tensor([.1,1.]), gpr_noise_bounds = torch.tensor([0.001, .01]), \
                              cp_bounds = torch.tensor([0., 1.]), gpr_bias_bounds = torch.tensor([-2., 2.])):
    # print('starting')
    noise=torch.tensor(1E-5)
    jitter=torch.tensor(1E-5)
    
    Mf = yf.shape[1]
    Nf = xf.shape[0]
    Ns = xs.shape[0]
    Nsf = Ns + Nf
    Xsf = torch.vstack((xs,xf))
    
    changepoint_min_bound = cp_bounds[0]*torch.ones((num_regions-1))
    changepoint_max_bound = cp_bounds[1]*torch.ones((num_regions-1))
    gpr_var_min_bound = gpr_var_bounds[0]*torch.ones((num_regions,Mf))
    gpr_var_max_bound = gpr_var_bounds[1]*torch.ones((num_regions,Mf))
    gpr_ls_min_bound = gpr_ls_bounds[0]*torch.ones((num_regions,Mf))
    gpr_ls_max_bound = gpr_ls_bounds[1]*torch.ones((num_regions,Mf))
    gpr_bias_min_bound = gpr_bias_bounds[0]*torch.ones((num_regions,Mf))
    gpr_bias_max_bound = gpr_bias_bounds[1]*torch.ones((num_regions,Mf))
    
    changepoint = pyro.sample('changepoint', dist.Uniform(changepoint_min_bound,changepoint_max_bound ) )
    gpr_noise = pyro.sample("gpr_noise", dist.Uniform(gpr_noise_bounds[0], gpr_noise_bounds[1]))
    gpr_var = pyro.sample("gpr_var", dist.Uniform(gpr_var_min_bound, gpr_var_max_bound))
    gpr_lengthscale = pyro.sample("gpr_lengthscale", dist.Uniform(gpr_ls_min_bound, gpr_ls_max_bound))
    gpr_bias = pyro.sample("gpr_bias", dist.Uniform(gpr_bias_min_bound, gpr_bias_max_bound))
    
    region_labels = change_points_to_labels_torch(changepoint, Xsf)
    
    probs = one_hot(region_labels, num_regions)
    probs_fp = probs[Ns:,:]
    
    F = torch.zeros((Nf,num_regions,Mf))
    for j in range(Mf):
        for i in range(num_regions):
            with pyro.plate('latent_response' + str(i), Nf):
                eta = pyro.sample('sample'+str(i)+'_Mf'+str(j), dist.Normal(0, 1))
    
            f = compute_f_torch(gpr_var[i,j], gpr_lengthscale[i,j], gpr_bias[i,j], eta, xf)
            F[:,i,j] = f
    
    f_piecewise = torch.zeros((Nf, Mf))
    for j in range(Mf):
        for i in range(num_regions):
            f_piecewise[:,j] = f_piecewise[:,j] + probs_fp[:,i] * F[:,i,j]

    llk = dist.Categorical(probs=probs[:Ns,:]).log_prob(ys.flatten()).sum()  
    
    for j in range(Mf):
        llk = llk + dist.Normal(f_piecewise[:,j], torch.sqrt( gpr_noise ) ).log_prob(yf[:,j]).sum()
    
    pyro.deterministic("llk", llk)
    pyro.factor("obs", llk)

def model_SAGE_1D_PM_230628a(data):
    noise=torch.tensor(0.01)
    jitter=torch.tensor(1.0e-5)
    if not torch.is_tensor(data[0]):
        xs = torch.tensor(data[0])
        ys = torch.tensor(data[1])
        Xpred = torch.tensor(data[2])
        num_regions = torch.tensor(data[3])
    else:
        xs, ys, Xpred, num_regions = data   

    idx_st = torch.arange(xs.shape[0])
    
    bounds_min, bounds_max = bounds_set(num_regions, xs, ys)
    cp_ = pyro.sample('cp', dist.Uniform(bounds_min,bounds_max))
    
    cp_, _ = torch.sort(cp_)
    
    cluster_labels = change_points_to_labels_torch(cp_, xs)
    membership = one_hot(cluster_labels.long()).double()
    
    llk_pm = dist.Categorical(logits=membership[idx_st,:]).log_prob(ys.flatten().double()).sum()
    llk_pm = pyro.deterministic('llk_pm', llk_pm)
    return llk_pm

# ----- Prediction functions -----------
def predict_SAGE_1D_230628a(samples, i, xs, Xnew, num_regions, xf, yf, eps = 1E-6):   
    Nnew = Xnew.shape[0]
    Mf = yf.shape[1]
    gpr_new_mean_regions = torch.zeros((Nnew,Mf,num_regions))
    gpr_new_var_regions = torch.zeros((Nnew,Mf,num_regions))
    gpr_new_cov_regions = torch.zeros((Nnew,Nnew,Mf,num_regions))
    gpr_new_mean_mixture = torch.zeros((Nnew,Mf))
    gpr_new_var_mixture = torch.zeros((Nnew,Mf))
    
    region_labels = change_points_to_labels_torch(samples['changepoint'][i], Xnew)
    probs = one_hot(region_labels, num_regions)
    
    # get gpr
    # gpr samples: 'sample'+str(i)+'_Mf'+str(j) , i is num region, j is functional property index
    
    F = torch.zeros((Nnew,num_regions,Mf))
    V = torch.zeros((Nnew,num_regions,Mf))
    for k in range(Mf):
        for j in range(num_regions):
            eta = samples['sample'+str(j)+'_Mf'+str(k)][i]
            f = compute_f_torch(samples['gpr_var'][i,j,k],
                                samples['gpr_lengthscale'][i,j,k],
                                samples['gpr_bias'][i,j,k], eta, xf)
            mean, _, var = gpr_forward_torch(samples['gpr_var'][i,j,k],
                                         samples['gpr_lengthscale'][i,j,k],
                                         xf,f, Xnew, samples['gpr_noise'][i], include_noise=False)
            F[:,j,k] = mean   
            V[:,j,k] = var
    
    f_piecewise = torch.zeros((Nnew, Mf, 1))
    v_piecewise = torch.zeros((Nnew, Mf, 1))
    f_sample = torch.zeros((Nnew, Mf,1))
    for k in range(Mf):
        for j in range(num_regions):
            f_piecewise[:,k,0] = f_piecewise[:,k,0] + probs[:,j] * F[:,j,k]
            v_piecewise[:,k,0] = v_piecewise[:,k,0] + probs[:,j] * V[:,j,k]
        f_sample[:,k,0] = dist.Normal(f_piecewise[:,k,0], torch.sqrt( samples['gpr_noise'][i] ) ).sample()

    # return np.array(probs), np.array(gpr_new_mean_mixture), np.array(gpr_new_var_mixture)
    return np.array(probs), np.array(f_piecewise), np.array(f_sample), np.array(F), np.array(v_piecewise)

def predict_SAGE_1D_FP_230628a(samples, i, Xnew, num_regions, xf, yf, eps = 1E-6):   
    Nnew = Xnew.shape[0]
    Mf = yf.shape[1]
    gpr_new_mean_regions = torch.zeros((Nnew,Mf,num_regions))
    gpr_new_var_regions = torch.zeros((Nnew,Mf,num_regions))
    gpr_new_cov_regions = torch.zeros((Nnew,Nnew,Mf,num_regions))
    gpr_new_mean_mixture = torch.zeros((Nnew,Mf))
    gpr_new_var_mixture = torch.zeros((Nnew,Mf))
    
    region_labels = change_points_to_labels_torch(samples['changepoint'][i], Xnew)
    probs = one_hot(region_labels, num_regions)
    
    # get gpr
    # gpr samples: 'sample'+str(i)+'_Mf'+str(j) , i is num region, j is functional property index
    
    F = torch.zeros((Nnew,num_regions,Mf))
    V = torch.zeros((Nnew,num_regions,Mf))
    for k in range(Mf):
        for j in range(num_regions):
            eta = samples['sample'+str(j)+'_Mf'+str(k)][i]
            f = compute_f_torch(samples['gpr_var'][i,j,k],
                                samples['gpr_lengthscale'][i,j,k],
                                samples['gpr_bias'][i,j,k], eta, xf)
            mean, _, var = gpr_forward_torch(samples['gpr_var'][i,j,k],
                                         samples['gpr_lengthscale'][i,j,k],
                                         xf,f, Xnew, samples['gpr_noise'][i], include_noise=False)
            F[:,j,k] = mean   
            V[:,j,k] = var
    
    f_piecewise = torch.zeros((Nnew, Mf, 1))
    v_piecewise = torch.zeros((Nnew, Mf, 1))
    f_sample = torch.zeros((Nnew, Mf,1))
    for k in range(Mf):
        for j in range(num_regions):
            f_piecewise[:,k,0] = f_piecewise[:,k,0] + probs[:,j] * F[:,j,k]
            v_piecewise[:,k,0] = v_piecewise[:,k,0] + probs[:,j] * V[:,j,k]
        f_sample[:,k,0] = dist.Normal(f_piecewise[:,k,0], torch.sqrt( samples['gpr_noise'][i] ) ).sample()

    # return np.array(probs), np.array(gpr_new_mean_mixture), np.array(gpr_new_var_mixture)
    return np.array(probs), np.array(f_piecewise), np.array(f_sample), np.array(F), np.array(v_piecewise)

def preds_to_outputs(preds, samples, data = None, plot = False):
    gpc_new_mean = np.stack([item[0] for item in preds]).mean(axis=0)
    gpc_new_std = np.stack([item[0] for item in preds]).std(axis=0)

    # squeeze assumes 1 functional property:
    gpr_samples_noiseless = np.stack([item[1] for item in preds], axis = 2).squeeze(axis = 1)
    gpr_samples = np.stack([item[2] for item in preds], axis = 2).squeeze(axis = 1)
    gpr_regions = np.stack([item[3] for item in preds], axis = 2)
    
    # assume Mf == 1
    gpr_new_mean = gpr_samples.mean(axis = 1)
    gpr_new_var = gpr_samples.std(axis = 1)**2
    
    labels_new_est = gpc_new_mean #np.mean(gpc_new_mean,axis=1)
    labels_new_entropy = entropy(gpc_new_mean, axis =1)

    cp_samples = np.array(samples['changepoint'].flatten())
    
    results = [gpc_new_mean, gpc_new_std, gpr_new_mean, \
    gpr_new_var, labels_new_est, labels_new_entropy, gpr_regions, gpr_samples_noiseless, cp_samples]
    
    if plot:
        xs = data[0].detach().numpy()
        ys = data[1].detach().numpy()
        xf = data[2].detach().numpy()
        yf = data[3].detach().numpy()
        Xnew = data[4].detach().numpy()
        num_regions = data[5]
    
        plt.figure(figsize=(7,3), dpi=300)
        plt.subplot(1,num_regions,1)
        plt.plot(Xnew, labels_new_est)

        plt.subplot(1,num_regions,2)
        plt.plot(Xnew, labels_new_entropy)

        plt.plot(xs, ys, 'kd')
        plt.plot(xf, yf,'ks')

        plt.figure()
        # for j in range(yf.shape[1]): # for each functional property
        k = 0
        plt.plot(Xnew, gpr_new_mean[:,k]) 
        plt.fill_between(Xnew.flatten(), gpr_new_mean[:,k].flatten() - 1.96*np.sqrt(gpr_new_var[:,k].flatten()),
                     gpr_new_mean[:,k].flatten() + 1.96*np.sqrt(gpr_new_var[:,k].flatten()), alpha=0.5)
        plt.plot(xs, ys, 'kd')
        plt.plot(xf, yf,'ks')

        plt.figure(figsize = (7,3))
        plt.subplot(1,2,1)
        # for j in range(yf.shape[1]): # for each functional property
        k = 0
        plt.plot(Xnew, gpr_new_mean[:,k]) 
        plt.fill_between(Xnew.flatten(), gpr_new_mean[:,k].flatten() - 1.96*np.sqrt(gpr_new_var[:,k].flatten()),
                     gpr_new_mean[:,k].flatten() + 1.96*np.sqrt(gpr_new_var[:,k].flatten()), alpha=0.5)
        plt.plot(xs, ys, 'kd')
        plt.plot(xf, yf,'ks')
        plt.plot(Xnew, gpr_samples_noiseless.squeeze().reshape((gpr_samples_noiseless.shape[0],gpr_samples_noiseless.shape[1]*gpr_samples_noiseless.shape[2])));

        plt.subplot(1,2,2)
        plt.plot(Xnew, gpr_samples.squeeze().reshape((gpr_samples.shape[0],gpr_samples.shape[1]*gpr_samples.shape[2])));

        plt.figure()
        plt.hist(np.array(samples['changepoint'].flatten()))
        plt.title('cp')

        color_num = np.linspace(0,1, yf.shape[1])
        plt.figure(figsize=(10,3))
        for j in range(num_regions):
            plt.subplot(1,num_regions,j+1)
            for k in range(yf.shape[1]):
                plt.hist(np.array(samples['gpr_var'][:,j,k].flatten()),bins=20,color=plt.cm.viridis(color_num[k]), alpha=.5)
            plt.title('gpr var fp region' + str(j))
            plt.legend(['fp'+str(i) for i in range(yf.shape[1])])

        plt.figure(figsize=(10,3))
        for j in range(num_regions):
            plt.subplot(1,num_regions,j+1)
            for k in range(yf.shape[1]):
                plt.hist(np.array(samples['gpr_lengthscale'][:,j,k].flatten()),bins=20, alpha=.5)
            plt.title('gpr ls fp region' + str(j))
            plt.legend(['fp'+str(i) for i in range(yf.shape[1])])
        
    return results

# -- plotting ----------  
def plot_uq_SAGE_1D_230628a(xs, ys, xf, yf, Xnew, num_regions, results):
    # assumes 1 functional property
    gpc_mean = results['gpc_mean']
    gpc_std = results['gpc_std']
    gpr_mean = results['gpr_mean']
    gpr_var = results['gpr_var']
    
    gpr_samples_noiseless = results['gpr_samples_noiseless']
    gpr_samples = results['gpr_samples']
    
    cp_samples = results['cp_samples']
    samples = results['samples']
    
    labels_est = gpc_mean
    labels_entropy = entropy(gpc_mean, axis =1)
    
    plt.figure(figsize=(7,3), dpi=300)
    plt.subplot(1,num_regions,1)
    plt.plot(Xnew, labels_est)
    
    plt.subplot(1,num_regions,2)
    plt.plot(Xnew, labels_entropy)
    
    plt.plot(xs, ys, 'kd')
    plt.plot(xf, yf,'ks')

    plt.figure()
    k = 0
    plt.plot(Xnew, gpr_mean[:,k]) 
    plt.fill_between(Xnew.flatten(), gpr_mean[:,k].flatten() - 1.96*np.sqrt(gpr_var[:,k].flatten()),
                 gpr_mean[:,k].flatten() + 1.96*np.sqrt(gpr_var[:,k].flatten()), alpha=0.5)
    plt.plot(xs, ys, 'kd')
    plt.plot(xf, yf,'ks')
    
    plt.figure(figsize = (7,3))
    plt.subplot(1,2,1)
    k = 0
    plt.plot(Xnew, gpr_mean[:,k]) 
    plt.fill_between(Xnew.flatten(), gpr_mean[:,k].flatten() - 1.96*np.sqrt(gpr_var[:,k].flatten()),
                 gpr_mean[:,k].flatten() + 1.96*np.sqrt(gpr_var[:,k].flatten()), alpha=0.5)
    plt.plot(xs, ys, 'kd')
    plt.plot(xf, yf,'ks')
    plt.plot(Xnew, gpr_samples_noiseless.squeeze().reshape((gpr_samples_noiseless.shape[0],gpr_samples_noiseless.shape[1]*gpr_samples_noiseless.shape[2])));
    
    plt.subplot(1,2,2)
    plt.plot(Xnew, gpr_samples.squeeze().reshape((gpr_samples.shape[0],gpr_samples.shape[1]*gpr_samples.shape[2])));

    plt.figure()
    plt.hist(np.array(cp_samples.flatten()))
    plt.title('cp')
    
    color_num = np.linspace(0,1, yf.shape[1])
    plt.figure(figsize=(10,3))
    for j in range(num_regions):
        plt.subplot(1,num_regions,j+1)
        for k in range(yf.shape[1]):
            plt.hist(np.array(samples['gpr_var'][:,j,k].flatten()),bins=20,color=plt.cm.viridis(color_num[k]), alpha=.5)
        plt.title('gpr var fp region' + str(j))
        plt.legend(['fp'+str(i) for i in range(yf.shape[1])])
        
    plt.figure(figsize=(10,3))
    for j in range(num_regions):
        plt.subplot(1,num_regions,j+1)
        for k in range(yf.shape[1]):
            plt.hist(np.array(samples['gpr_lengthscale'][:,j,k].flatten()),bins=20, alpha=.5)
        plt.title('gpr ls fp region' + str(j))
        plt.legend(['fp'+str(i) for i in range(yf.shape[1])])

# --- GP ------- 
def gpr_1D_change_point_discovery_gpflow(xf, yf, Xpred, numcp, gpkernel):
    start_kerns = []
    for i in range(numcp+1):
        if gpkernel == 'RBF':
            start_kerns.append( gpflow.kernels.RBF(lengthscales = .2) )
        elif gpkernel == 'Matern32':
            start_kerns.append( gpflow.kernels.Matern32() )

    cp = np.linspace(Xpred.min(),Xpred.max(),numcp+2)
    cp = cp[1:-1]
    k_ = gpflow.kernels.ChangePoints(start_kerns, cp, steepness=100.0)

    lik = gpflow.likelihoods.Gaussian()
    meanf = gpflow.mean_functions.Constant(np.mean(yf))
    m = gpflow.models.VGP((xf, yf), kernel=k_, likelihood=lik, mean_function = meanf) # now build the GP model as normal

    llk = m.predict_log_density((xf,yf)).numpy()
    
    m.likelihood.variance.assign(0.01)
        
#     m.kern.lengthscales.prior = tfp.distributions.Uniform(.1,10.)
#     m.kern.variance.prior = tfp.distributions.Uniform(.01,1.)
    
    gpflow.utilities.set_trainable(m.likelihood.variance, False)
#     p = m.likelihood.variance
#     m.likelihood.variance = gpflow.Parameter(p, transform=tfp.bijectors.Sigmoid(f64(0.0001), f64(0.01)) )            

    #maxiter = ci_niter(10000)
    gpflow.optimizers.Scipy().minimize(m.training_loss, m.trainable_variables, options=dict(maxiter=10000), method="tnc",) # fit the covariance function parameters

    yy, uy = m.predict_f(Xpred) 
    Fmu = yy.numpy().flatten()
    Fvar = uy.numpy().flatten()
    Yvar = Fvar + m.likelihood.variance.numpy()
    
    results = {'gpr_mean':Fmu, 'gpr_var':Fvar, 'llk':llk, 'cp':m.kernel.locations}
    return results

def compute_f_torch(variance, lengthscales, bias, eta, X):
    N = X.shape[0]
    K = RBF_torch(variance, lengthscales, X) + torch.eye(N) * 1e-6
    L = torch.linalg.cholesky(K)
    return torch.matmul(L, eta) + bias

def gpr_forward_torch(variance,lengthscales,xtrain,ytrain,xnew,noise_var,include_noise = True, prob_weights = None, jitter = 1E-6):
    # n is new, t is train    
    ytrain = ytrain.flatten()
    
    K_nt = RBF_torch(variance, lengthscales, xnew, xtrain)
    K_tt = RBF_torch(variance, lengthscales, xtrain, xtrain)
    K_nn = RBF_torch(variance, lengthscales, xnew, xnew)
    
    I_noise = torch.eye(K_tt.shape[0])*noise_var
    L = torch.linalg.inv(K_tt + I_noise)
    
    if prob_weights is None:
        mean = torch.matmul(K_nt,torch.matmul(L,ytrain[:,None]))
    else:
        fit_mean = torch.sum(ytrain*prob_weights.flatten()) / torch.sum(prob_weights.flatten())
        ytrain = ytrain - fit_mean
        mean = torch.matmul(K_nt,torch.matmul(L,ytrain[:,None])) + fit_mean
        
    # new y. Take ytrain - sum(ytrain*prob[i])/sum(prob[i])
    
    cov = K_nn - torch.matmul(K_nt, torch.matmul(L,K_nt.T) )
    cov = cov + torch.eye(cov.shape[0])*jitter
    if include_noise:
        cov = cov + torch.eye(cov.shape[0])*noise_var
    var = torch.diagonal(cov)
    return mean.flatten(), cov, var.flatten()

def RBF_torch(variance, lengthscales, X, Z = None):
    # built from: https://github.com/pyro-ppl/pyro/blob/727aff741e105715840bfdafee5bfeda7e8b65e8/pyro/contrib/gp/kernels/isotropic.py#L41
    if Z is None:
        Z = X
#     if jnp.isscalar(lengthscales):
#         lengthscales = lengthscales*jnp.ones((2))
    scaled_X = X / lengthscales
    scaled_Z = Z / lengthscales
    X2 = (scaled_X**2).sum(1, keepdims=True)
    Z2 = (scaled_Z**2).sum(1, keepdims=True)
    XZ = torch.matmul(scaled_X, scaled_Z.T)
    r2 = X2 - 2 * XZ + Z2.T
    return variance * torch.exp(-0.5 * r2)
 
def kernel(X, Z, var, length, noise, jitter=1.0e-6, include_noise=True):
    deltaXsq = torch.pow((X.double() - Z.T.double()) / length.double(), 2.0)
    k = var.double() * torch.exp(-0.5 * deltaXsq).double()
    if include_noise:
        k += (noise.double() + jitter) * torch.eye(X.shape[0]).double()
    return k

def gpkernel(X, Z, var, length, noise, jitter=1.0e-6, include_noise=True):
    deltaXsq = torch.pow((X.double() - Z.T.double()) / length.double(), 2.0)
    k = var.double() * torch.exp(-0.5 * deltaXsq).double()
    if include_noise:
        k += (noise.double() + jitter) * torch.eye(X.shape[0]).double()
    return k

def gp_forward(Xtest, Xtrain, Ytrain, var, lengthscale, noise):
    # Derived from: https://num.pyro.ai/en/0.7.1/examples/gp.html
    k_pp = gpkernel(Xtest, Xtest, var, lengthscale, noise, include_noise=True)
    k_pX = gpkernel(Xtest, Xtrain, var, lengthscale, noise, include_noise=False)
    k_XX = gpkernel(Xtrain, Xtrain, var, lengthscale, noise, include_noise=True)

    #k_XX = torch.tensor( nearestPD(k_XX.detach().numpy()) ).double()
    K_xx_inv = torch.linalg.inv(k_XX.double()).double()
    # print(k_XX.shape, K_xx_inv.shape, Xtest.shape, Xtrain.shape, Ytrain.shape)
    K_xx_post = k_pp.double() - torch.matmul(k_pX.double(), torch.matmul(K_xx_inv.double(), k_pX.T.double())).double()
    mean_xx_post = torch.matmul(k_pX.double(), torch.matmul(K_xx_inv.double(), Ytrain.double()))
    return mean_xx_post, K_xx_post

# --- Other support functions -------
def bounds_set_csp(num_regions, xs, ys):
    xs = xs.flatten()
    ys = ys.flatten()
    bounds_min = torch.ones((num_regions-1,1))*xs.min()
    bounds_max = torch.ones((num_regions-1,1))*xs.max()
    uL = torch.unique(ys)
    if uL.shape[0] == 2:
        bounds_min = torch.ones((num_regions-1,1))*xs[ys == 0].max()
        idx_y_is_1_or_2 = torch.logical_or(ys == 1, ys == 2)
        bounds_max = torch.ones((num_regions-1,1))*xs[idx_y_is_1_or_2].min()
    if uL.shape[0] > 2:
        bounds_points = torch.zeros((uL.shape[0],2))
        bounds_min = []
        bounds_max = []
        for i in range(uL.shape[0]):
            bounds_points[i,:] = torch.tensor([xs[ys == uL[i]].min(), xs[ys == uL[i]].max()])
        sorted_bounds, _ = torch.sort(bounds_points.flatten())
        
        for i in range(1, sorted_bounds.shape[0]-2, 2):
            bounds_min.append(sorted_bounds[i])
            bounds_max.append(sorted_bounds[i+1])

    bounds_min = torch.tensor(bounds_min)
    bounds_max = torch.tensor(bounds_max)
    return bounds_min, bounds_max

def bounds_set(num_regions, xs, ys):
    bounds_min = torch.ones((num_regions-1,1))*xs.min()
    bounds_max = torch.ones((num_regions-1,1))*xs.max()
    uL = torch.unique(ys)
    if uL.shape[0] == 2:
        bounds_min = torch.ones((num_regions-1,1))*xs[ys == 0].max()
        bounds_max = torch.ones((num_regions-1,1))*xs[ys == 1].min()
    if uL.shape[0] > 2:
        bounds_points = torch.zeros((uL.shape[0],2))
        bounds_min = []
        bounds_max = []
        for i in range(uL.shape[0]):
            bounds_points[i,:] = torch.tensor([xs[ys == uL[i]].min(), xs[ys == uL[i]].max()])
        sorted_bounds, _ = torch.sort(bounds_points.flatten())
        # sorted_bounds = [0min 0max 1min 1max 2min 2max]
        for i in range(1, sorted_bounds.shape[0]-2, 2):
            bounds_min.append(sorted_bounds[i])
            bounds_max.append(sorted_bounds[i+1])
        bounds_min = torch.tensor(bounds_min)
        bounds_max = torch.tensor(bounds_max)
    if num_regions>2:
        bounds_min=bounds_min[:,None]
        bounds_max=bounds_max[:,None]
    return bounds_min, bounds_max

def cp_samples_to_class_prob(cp_, X):
    # print(cp_.shape, X.shape)
    if type(X) is not np.ndarray:
        X = X.numpy()
    # cl_ = np.zeros((X.shape[0],cp_.shape[0]))
    one_hot_samples = np.zeros((cp_.shape[0],X.shape[0],cp_.shape[1]+1))
    # print(one_hot_samples.shape)
    for i in range(cp_.shape[0]):
        cl = change_points_to_labels(cp_[i,:], X)
        # print(cp_[i,:], np.unique(cl))
        one_hot_samples[i,:,:] = one_hot_np(cl, cp_[i,:], X)
    probs = np.mean(one_hot_samples, axis = 0)
    return probs

def change_points_to_labels(cp, X):
    if type(X) is not np.ndarray:
        X = X.numpy()
    cp = np.sort(cp)
    cl = np.zeros((X.shape[0])).astype(np.compat.long)
    N = cp.shape[0] # N = 3
    for i in np.arange(0, N):
        if i < N-1:
            idx = np.logical_and(X > cp[i], X < cp[i+1])
        elif i == N-1:
            idx = X > cp[i]
        cl[idx.flatten()] = i+1
    return cl

def change_points_to_labels_torch(cp, X):
    if type(X) is np.ndarray:
        X = torch.tensor(X)
    cp,_ = torch.sort(cp)
    cl = torch.zeros((X.shape[0])).long()
    N = cp.shape[0] # N = 3
    for i in range(0, N):
        if i < N-1:
            idx = torch.logical_and(X > cp[i], X < cp[i+1])
        elif i == N-1:
            idx = X > cp[i]
        cl[idx.flatten()] = i+1
    return cl

def one_hot_cp_np(v, cp, X):
    oh = np.zeros((v.shape[0], v.max()+1))
    # print(oh.shape)
    oh[np.arange(v.shape[0]),v] = 1
    if cp.max() > cp.min():
        if cp.min() < X.min():
            oh = np.hstack((oh,np.zeros((v.shape[0],1))))
        if cp.max() > X.max():
            oh = np.hstack((np.zeros((v.shape[0],1)),oh))  
        if not np.any( np.logical_and( cp.min() < X.flatten(), cp.max() > X.flatten()) ):
        # if no X are between the changepoints, add a middle column to one_hot
            oh = np.hstack(( oh[:,0][:,None], np.zeros((v.shape[0],1)), oh[:,1][:,None] ))
    # print(oh.shape)
    return oh

def one_hot_np(v, cp, X):
    oh = np.zeros((v.shape[0], cp.shape[0] + 1))
    oh[np.arange(v.shape[0]),v] = 1
    return oh

def torch_to_tf(x):
    if type(x) is np.ndarray:
        out = tf.convert_to_tensor(x.astype(np.double))
    else:
        out = tf.convert_to_tensor( x.numpy().astype(np.double) )
    return out
        
def to_numpy(v):
    for i in range(len(v)):
        if type(v[i]) is not np.ndarray:
            v[i] = v[i].numpy()
    return v

def to_torch(v):
    if not torch.is_tensor(v):
        v = torch.tensor(v)
    return v

### 1D Edge Case Challenges.

#### Set up 1D Challenges data
- Challenge 1: Structure data is more informative of phase boundaries.
- Challenge 2: Functional property is more informative of phase boundaries.

In [ ]:
x = np.linspace(0,1,100)

y = simple_challenge_1D_2regions(x)
plt.figure()
plt.plot(x,y,'k.')

# Actual X values
xx = np.linspace(0,1,100)
xst1 = np.asarray([0., 0.65, 0.75, 1.])[:,None]
xfp1 = np.asarray([0.,.2, .4, .5, .8, .9, 1.])[:,None]

xst2 = np.asarray([0., .4, .8, 1.])[:,None]
xfp2 = np.linspace(0,1,20)[:,None]

yyst = xx > 0.7
yst1 = xst1 > 0.7
yfp1 = simple_challenge_1D_2regions(xfp1)
yst2 = xst2 > 0.7
yfp2 = simple_challenge_1D_2regions(xfp2)

plt.figure(figsize = (9,3))
plt.subplot(1,2,1)
plt.plot(x, y, 'k--')
plt.plot(xfp1, yfp1, 'ks')
plt.plot(xst1, simple_challenge_1D_2regions(xst1), 'rd')
plt.subplot(1,2,2)
plt.plot(x, y, 'k--')
plt.plot(xfp2, yfp2, 'ks')
plt.plot(xst2, simple_challenge_1D_2regions(xst2), 'rd')

fig = plt.figure(figsize = (4,3), dpi=300)
ax11 = fig.add_subplot()
ax11.plot(x, y, 'k--')
plt.plot(xfp1, yfp1, 'ks')
ax11.spines['top'].set_visible(False)
ax11.set_ylabel('functional property')
ax12 = ax11.twinx()
ax12.plot(xx, yyst, 'r:')
ax12.plot(xst1, yst1, 'rd')
ax12.set_yticks([0,1])
ax12.spines['right'].set_color('red')
ax12.tick_params(axis='y', colors='red')
ax12.spines['top'].set_visible(False)
ax12.set_ylabel('phase region label', rotation = 270, color = 'red')

fig = plt.figure(figsize = (4,3), dpi=300)
ax21 = fig.add_subplot()
ax21.plot(x, y, 'k--')
plt.plot(xfp2, yfp2, 'ks')
ax21.spines['top'].set_visible(False)
ax21.set_ylabel('functional property')
ax22 = ax21.twinx()
ax22.plot(xx, yyst, 'r:')
ax22.plot(xst2, yst2, 'rd')
ax22.set_yticks([0,1])
ax22.spines['right'].set_color('red')
ax22.tick_params(axis='y', colors='red')
ax22.spines['top'].set_visible(False)
ax22.set_ylabel('phase region label', rotation = 270, color = 'red')

N = x.shape[0]
XY = np.concatenate((x[:,None],np.zeros((N,1))),axis=1)
S = np.diag(np.ones(N-1),-1) + np.diag(np.ones(N-1),1)
plt.figure(figsize = (10,10))
plot_graph(S, XY)

#### Run algorithms

In [ ]:
# 2 Regions

# Run Segmentation and GPR-CP on each.
numcp = 1
Xpred = np.linspace(0,1,100)[:,None]
num_regions = 2
lslimits = [1.,10.]

starting_data = [Xpred, x, y, yyst]
data_ch1 = [xst1, yst1, xfp1, yfp1]
data_ch2 = [xst2, yst2, xfp2, yfp2]


# run changepoint detection using gpr with changepoint kernel.
# gpr_fmean_ch1, gpr_fvar_ch1, _, _, gpr_model_ch1
results_gpr_ch1 = gpr_1D_change_point_discovery_gpflow(xfp1, yfp1, Xpred, numcp, 'RBF')
results_gpr_ch2 = gpr_1D_change_point_discovery_gpflow(xfp2, yfp2, Xpred, numcp, 'RBF')


# run changepoint detection using only structure data
numsteps = 1000
# cat_dist_ch1, _, cp_ch1, _, max_llk_cp_ch1 
results_SAGE_1D_PM_ch1, posterior_PM_ch1 = SAGE_1D_PM_230628a(Xpred, xst1, yst1, num_regions, numsteps = numsteps)
results_SAGE_1D_PM_ch2, posterior_PM_ch2 = SAGE_1D_PM_230628a(Xpred, xst2, yst2, num_regions, numsteps = numsteps)


num_samples = 1000
# Pmufp1, Pvarfp1, mean_unifiedfp1, var_unifiedfp1, cpfp1, mllk_meanfp1, mllk_varfp1, max_llk_sample_cpfp1
results_SAGE_1D_FP_ch1 = SAGE_1D_FP_230628a(xfp1, yfp1, Xpred, num_regions = 2, num_samples = num_samples, num_warmup = 100, 
                                      gpr_var_bounds = torch.tensor([.01, 2.]), gpr_ls_bounds = torch.tensor([.2,1.]),
                                       gpr_noise_bounds = torch.tensor([0.001,.01]), cp_bounds = torch.tensor([0.5, 1.]))
results_SAGE_1D_FP_ch2 = SAGE_1D_FP_230628a(xfp2, yfp2, Xpred, num_regions = 2, num_samples = num_samples, num_warmup = 100, 
                                      gpr_var_bounds = torch.tensor([.01, 2.]), gpr_ls_bounds = torch.tensor([.2,1.]),
                                       gpr_noise_bounds = torch.tensor([0.001,.01]), cp_bounds = torch.tensor([0.5, 1.]))


# Pmuj0, Pvarj0, mean_unified0, var_unified0, cpj0, mllk_meanj0, mllk_varj0, max_llk_sample_cp0
results_SAGE_1D_ch1 = SAGE_1D_230628a(xst1, yst1, xfp1, yfp1, Xpred, num_regions = 2, num_samples = num_samples, num_warmup = 100, 
                                      gpr_var_bounds = torch.tensor([.01, 2.]), gpr_ls_bounds = torch.tensor([.2,1.]),
                                       gpr_noise_bounds = torch.tensor([0.001,.01]), cp_bounds = torch.tensor([0.5, 1.]))
results_SAGE_1D_ch2 = SAGE_1D_230628a(xst2, yst2, xfp2, yfp2, Xpred, num_regions = 2, num_samples = num_samples, num_warmup = 100, 
                                      gpr_var_bounds = torch.tensor([.01, 2.]), gpr_ls_bounds = torch.tensor([.2,1.]),
                                       gpr_noise_bounds = torch.tensor([0.001,.01]), cp_bounds = torch.tensor([0.5, 1.]))

plot_uq_SAGE_1D_230628a(xst1, yst1, xfp1, yfp1, Xpred, num_regions, results_SAGE_1D_ch1)


data = {'starting_data':starting_data, 'challenge_data':[data_ch1, data_ch2],
        'gpr':[results_gpr_ch1, results_gpr_ch2],
        'SAGE_1D_PM':[results_SAGE_1D_PM_ch1, results_SAGE_1D_PM_ch2],
        'SAGE_1D_FP':[results_SAGE_1D_FP_ch1, results_SAGE_1D_FP_ch2],
        'SAGE_1D':[results_SAGE_1D_ch1, results_SAGE_1D_ch2]}

with open(r"1D_results_231031b.dill", "wb") as output_file:
    dill.dump(data, output_file)

#### Visualize Results

In [ ]:
def plot_3_predictions(data, idx):
    fig = plt.figure(figsize = (4,3), dpi=300)
    Xpred, x, y, yyst = data['starting_data']
    xst, yst, xfp, yfp = data['challenge_data'][idx]
    ax11 = fig.add_subplot()
    ax11.plot(x, y, 'k--')
    ax11.plot(xfp, yfp, 'ks')
    ax11.spines['top'].set_visible(False)
    ax11.set_ylabel('functional property')
    ax12 = ax11.twinx()
    ax12.plot(xx, yyst, 'r:')
    ax12.plot(xst, yst, 'rd')
    ax12.set_yticks([-0.2,0,1,1.2])
    ax12.spines['right'].set_color('red')
    ax12.tick_params(axis='y', colors='red')
    ax12.spines['top'].set_visible(False)
    ax12.set_ylabel('phase region label', rotation = 270, color = 'red')    
    
    # plot gpr model using changepoint kernel
    ax11.plot(Xpred, data['gpr'][idx]['gpr_mean'], 'tab:blue');
    ax11.fill_between(Xpred.flatten(), data['gpr'][1]['gpr_mean'].flatten() - 1.96*np.sqrt(data['gpr'][idx]['gpr_var'].flatten()),
                     data['gpr'][idx]['gpr_mean'].flatten() + 1.96*np.sqrt(data['gpr'][idx]['gpr_var'].flatten()), alpha=0.5, color = 'tab:blue')

    # plot gpr from joint model 
    ax11.plot(Xpred, data['SAGE_1D'][idx]['gpr_mean'], 'tab:green');
    ax11.fill_between(Xpred.flatten(), data['SAGE_1D'][idx]['gpr_mean'].flatten() - 1.96*data['SAGE_1D'][idx]['gpr_std'].flatten(),
                     data['SAGE_1D'][idx]['gpr_mean'].flatten() + 1.96*data['SAGE_1D'][idx]['gpr_std'].flatten(), alpha=0.5, color = 'tab:green')
    # plt.title('blue:gpr w cp; green:joint')

    # MAX LLK
    ax11.plot(Xpred, data['SAGE_1D'][idx]['maxllk_gpr_fmean'], 'tab:purple');
    ax11.fill_between(Xpred.flatten(), data['SAGE_1D'][idx]['maxllk_gpr_fmean'].flatten() - 1.96*np.sqrt(data['SAGE_1D'][idx]['maxllk_gpr_fvar'].flatten()),
                     data['SAGE_1D'][idx]['maxllk_gpr_fmean'].flatten() + 1.96*np.sqrt(data['SAGE_1D'][idx]['maxllk_gpr_fvar'].flatten()), alpha=0.5, color = 'tab:purple')

    fig.savefig('1D_ex_' + 'ch'+str(idx+1) + '.png', bbox_inches="tight")

    # first case: plot changepoint distribution from structure model and joint model
    fig = plt.figure(figsize = (2,1), dpi = 300)
    plt.hist(data['SAGE_1D_PM'][idx]['cp_samples'], bins=20, color = 'tab:orange', alpha = 0.5);
    plt.hist(data['SAGE_1D'][idx]['cp_samples'],bins=20, color = 'tab:green', alpha = 0.5);

    fig.savefig('1D_cp_' + 'ch'+str(idx+1) + '.png', bbox_inches="tight")    
    
    
# plot data for first case
plot_3_predictions(data, 0)

# plot data for second case
plot_3_predictions(data, 1)


#### Performance calculations

In [ ]:
import dill
with open(r"1D_results_231031b.dill", "rb") as input_file:
    data = dill.load(input_file)

In [ ]:
import dill
import sklearn
import scipy
import tensorflow as tf
import gpflow
import numpy as np
f64 = gpflow.utilities.to_default_float
from gpflow.ci_utils import ci_niter
from sklearn.metrics import r2_score, accuracy_score, confusion_matrix

def tnp(x):
    return np.asarray(x).flatten()
    
with open(r"1D_results_231031b.dill", "rb") as input_file:
    data = dill.load(input_file)

Xpred, x, y_fp, y_st = data['starting_data']
xst1, yst1, xfp1, yfp1 = data['challenge_data'][0]
xst2, yst2, xfp2, yfp2 = data['challenge_data'][1]

xfp1_right = np.min(xfp1[xfp1 > .7])
xfp1_left = np.max(xfp1[xfp1 < .7])

print(xfp1_left, xfp1_right)

idx_kp = np.where(np.logical_or(Xpred < xfp1_left, Xpred > xfp1_right))[0]

sage_1D_PM_est_ch1 = Xpred > data['SAGE_1D_PM'][0]['cp_samples'].mean()
sage_1D_PM_est_ch2 = Xpred > data['SAGE_1D_PM'][1]['cp_samples'].mean()

sage_1D_FP_est_ch1 = Xpred > data['SAGE_1D_FP'][0]['cp_samples'].mean()
sage_1D_FP_est_ch2 = Xpred > data['SAGE_1D_FP'][1]['cp_samples'].mean()

sage_1D_joint_PM_est_ch1 = Xpred > data['SAGE_1D'][0]['cp_samples'].mean()
sage_1D_joint_PM_est_ch2 = Xpred > data['SAGE_1D'][1]['cp_samples'].mean()

gpr_cp_PM_est_ch1 = Xpred > data['gpr'][0]['cp']
gpr_cp_PM_est_ch2 = Xpred > data['gpr'][1]['cp']

# just PM EC1
C = 2
data_gpr = (tf.convert_to_tensor(xst1), tf.convert_to_tensor(yst1)) # create data variable that contains both the xy-coordinates of the currently measured samples and their labels.
kernel = gpflow.kernels.Matern52() #+ gpflow.kernels.White(variance=0.01)   # sum kernel: Matern32 + White
# Robustmax Multiclass Likelihood
invlink = gpflow.likelihoods.RobustMax(C)  # Robustmax inverse link function
likelihood = gpflow.likelihoods.MultiClass(C, invlink=invlink)  # Multiclass likelihood
m = gpflow.models.VGP(data=data_gpr, kernel=kernel, likelihood=likelihood, num_latent_gps=C) # set up the GP model

opt = gpflow.optimizers.Scipy() # set up the hyperparameter optimization
opt_logs = opt.minimize(m.training_loss, m.trainable_variables, method = 'tnc', options=dict(maxiter=ci_niter(1000)) ) # run the optimization
y = m.predict_y(tf.convert_to_tensor(Xpred)) # what is the Poisson process for the full XY coordinates
y_mean = y[0].numpy() # mean of y
y_var = y[1].numpy() # variance of y.
gpc_est_pm_ch1 = np.argmax(y_mean,axis=1)

# just PM EC2
C = 2
data_gpr = (tf.convert_to_tensor(xst2), tf.convert_to_tensor(yst2)) # create data variable that contains both the xy-coordinates of the currently measured samples and their labels.
kernel = gpflow.kernels.Matern52() #+ gpflow.kernels.White(variance=0.01)   # sum kernel: Matern32 + White
# Robustmax Multiclass Likelihood
invlink = gpflow.likelihoods.RobustMax(C)  # Robustmax inverse link function
likelihood = gpflow.likelihoods.MultiClass(C, invlink=invlink)  # Multiclass likelihood
m = gpflow.models.VGP(data=data_gpr, kernel=kernel, likelihood=likelihood, num_latent_gps=C) # set up the GP model

opt = gpflow.optimizers.Scipy() # set up the hyperparameter optimization
opt_logs = opt.minimize(m.training_loss, m.trainable_variables, method = 'tnc', options=dict(maxiter=ci_niter(1000)) ) # run the optimization
y = m.predict_y(tf.convert_to_tensor(Xpred)) # what is the Poisson process for the full XY coordinates
y_mean = y[0].numpy() # mean of y
y_var = y[1].numpy() # variance of y.
gpc_est_pm_ch2 = np.argmax(y_mean,axis=1)

r2_ch1_sage_joint = r2_score(y_fp[idx_kp],data['SAGE_1D'][0]['gpr_mean'][idx_kp])
r2_ch1_sage_fp = r2_score(y_fp[idx_kp],data['SAGE_1D_FP'][0]['gpr_mean'][idx_kp])
r2_ch1_gpr_cp = r2_score(y_fp[idx_kp],data['gpr'][0]['gpr_mean'][idx_kp])

r2_ch2_sage_joint = r2_score(y_fp[idx_kp],data['SAGE_1D'][1]['gpr_mean'][idx_kp])
r2_ch2_sage_fp = r2_score(y_fp[idx_kp],data['SAGE_1D_FP'][1]['gpr_mean'][idx_kp])
r2_ch2_gpr_cp = r2_score(y_fp[idx_kp],data['gpr'][1]['gpr_mean'][idx_kp])

y_st = tnp(y_st)
acc_ch1_sage_joint = accuracy_score(y_st, tnp(sage_1D_joint_PM_est_ch1))
acc_ch1_sage_st = accuracy_score(y_st, tnp(sage_1D_PM_est_ch1))
acc_ch1_sage_fp = accuracy_score(y_st, tnp(sage_1D_FP_est_ch1))
acc_ch1_gpr_cp = accuracy_score(y_st, tnp(gpr_cp_PM_est_ch1))
acc_ch1_gpc = accuracy_score(y_st, tnp(gpc_est_pm_ch1))

acc_ch2_sage_joint = accuracy_score(y_st, tnp(sage_1D_joint_PM_est_ch2))
acc_ch2_sage_st = accuracy_score(y_st, tnp(sage_1D_PM_est_ch2))
acc_ch2_sage_fp = accuracy_score(y_st, tnp(sage_1D_FP_est_ch2))
acc_ch2_gpr_cp = accuracy_score(y_st, tnp(gpc_est_pm_ch2))
acc_ch2_gpc = accuracy_score(y_st, tnp(gpc_est_pm_ch2))

f1s_ch1_sage_joint = sklearn.metrics.f1_score(y_st, tnp(sage_1D_joint_PM_est_ch1), average='micro')
f1s_ch1_sage_st = sklearn.metrics.f1_score(y_st, tnp(sage_1D_PM_est_ch1), average='micro')
f1s_ch1_sage_fp = sklearn.metrics.f1_score(y_st, tnp(sage_1D_FP_est_ch1), average='micro')
f1s_ch1_gpr_cp = sklearn.metrics.f1_score(y_st, tnp(gpr_cp_PM_est_ch1), average='micro')
f1s_ch1_gpc = sklearn.metrics.f1_score(y_st, tnp(gpc_est_pm_ch1), average='micro')

f1s_ch2_sage_joint = sklearn.metrics.f1_score(y_st, tnp(sage_1D_joint_PM_est_ch2), average='micro')
f1s_ch2_sage_st = sklearn.metrics.f1_score(y_st, tnp(sage_1D_PM_est_ch2), average='micro')
f1s_ch2_sage_fp = sklearn.metrics.f1_score(y_st, tnp(sage_1D_FP_est_ch2), average='micro')
f1s_ch2_gpr_cp = f1_score(y_st, tnp(gpc_est_pm_ch2), average='micro')
f1s_ch2_gpc = f1_score(y_st, tnp(gpc_est_pm_ch2), average='micro')

print('1D R2, SAGE:',r2_ch1_sage_joint, ' SAGE-FP:', r2_ch1_sage_fp, ' GPR_CP:', r2_ch1_gpr_cp)
print('1D Acc, SAGE:',acc_ch1_sage_joint, 'SAGE-PM:',acc_ch1_sage_st, 'SAGE-FP:',acc_ch1_sage_fp, 'GPR-CP:', acc_ch1_gpr_cp, ' GPC:',acc_ch1_gpc)
print('1D F1s, SAGE:',f1s_ch1_sage_joint, 'SAGE-PM:',f1s_ch1_sage_st, 'SAGE-FP:',f1s_ch1_sage_fp, 'GPR-CP:', f1s_ch1_gpr_cp, ' GPC:',f1s_ch1_gpc)
print('1D R2, SAGE:',r2_ch2_sage_joint, ' SAGE-FP:', r2_ch2_sage_fp, ' GPR_CP:', r2_ch2_gpr_cp)
print('1D Acc, SAGE:',acc_ch2_sage_joint, 'SAGE-PM:',acc_ch2_sage_st, 'SAGE-FP:',acc_ch2_sage_fp, 'GPR-CP:', acc_ch2_gpr_cp, ' GPC:',acc_ch2_gpc)
print('1D F1s, SAGE:',f1s_ch2_sage_joint, 'SAGE-PM:',f1s_ch2_sage_st, 'SAGE-FP:',f1s_ch2_sage_fp, 'GPR-CP:', f1s_ch2_gpr_cp, ' GPC:',f1s_ch2_gpc)
# plt.figure()
# plt.plot(Xpred, gpr_cp_mean1)
# plt.plot(Xpred, y_fp, 'r')
# plt.plot(Xpred, max_llk_sample_mean1)
# plt.plot(xfp1, yfp1, 'k.')


In [ ]:
### Added CAMEO:

import dill
import GPy
import sklearn
from sklearn.metrics import f1_score, r2_score
import scipy
import tensorflow as tf
import gpflow
f64 = gpflow.utilities.to_default_float
from gpflow.ci_utils import ci_niter
from jax.nn import one_hot as jax_one_hot
from cameo_240821a import *
import tensorflow_probability as tfp

from scipy.spatial import Voronoi
from scipy.spatial.distance import pdist, squareform
import applied_active_learning_191228a as al

with open(r"1D_results_231031b.dill", "rb") as input_file:
    data = dill.load(input_file)

Xpred, x, y_fp, y_st = data['starting_data']
xst1, yst1, xfp1, yfp1 = data['challenge_data'][0]
xst2, yst2, xfp2, yfp2 = data['challenge_data'][1]

print(xst1.shape, xfp1.shape)

# need indices for CAMEO inputs
kp_st1 = nearest_idx(Xpred, xst1)
kp_fp1 = nearest_idx(Xpred, xfp1)
kp_st2 = nearest_idx(Xpred, xst2)
kp_fp2 = nearest_idx(Xpred, xfp2)
N = Xpred.shape[0]
# Challenge 1----------------
# set up Cameo inputs
Ux = np.asarray(jax_one_hot(yst1*1,2)).squeeze()
S = np.diag(np.ones(N-1),-1) + np.diag(np.ones(N-1),1)
print(S.shape, Ux.shape, kp_st1.shape, kp_st1)
plt.figure()
cl_full, _ = GRF_applied(kp_st1, Ux, S)
cl_full= cl_full.flatten()
cl_fp = cl_full.flatten()[kp_fp1]
cameo_gpr_1a = np.zeros(Xpred.shape[0])

for i in range(2):
    k = gpflow.kernels.SquaredExponential(lengthscales = [1.])# + gpflow.kernels.White(variance=0.001) # set up kernel
    data = (tf.convert_to_tensor(xfp1[cl_fp==i,:]), tf.convert_to_tensor(yfp1[cl_fp==i].flatten()[:,None]))
    m = gpflow.models.GPR(data=data, kernel=k, mean_function=gpflow.mean_functions.Constant(yfp1[cl_fp==i].flatten().mean())) # set up GPR model
    
    m.likelihood.variance.assign(0.005)
    p = m.likelihood.variance
    m.likelihood.variance = gpflow.Parameter(p, transform=tfp.bijectors.Sigmoid(f64(0.001), f64(0.01)) )    
    
    opt = gpflow.optimizers.Scipy() # set up hyperparameter optimization
    opt_logs = opt.minimize(m.training_loss, m.trainable_variables, method = 'tnc', options=dict(maxiter=100))  # run optimization
    temp, _ = m.predict_f(tf.convert_to_tensor(Xpred[cl_full==i,:])) # compute the mean and variance for the other samples in the phase region
    cameo_gpr_1a[cl_full==i] = temp.numpy().flatten()

r2_2a_cameo = r2_score(y_fp,cameo_gpr_1a)
acc_2a_cameo = f1_score(y_st, cl_full)
print( r2_2a_cameo, acc_2a_cameo)

# Challenge 2----------------
Ux = np.asarray(jax_one_hot(yst2*1,2)).squeeze()
S = np.diag(np.ones(N-1),-1) + np.diag(np.ones(N-1),1)
plt.figure()
cl_full, _ = GRF_applied(kp_st2, Ux, S)
cl_full= cl_full.flatten()
cl_fp = cl_full.flatten()[kp_fp2]
cameo_gpr_1b = np.zeros(Xpred.shape[0])

for i in range(2):
    k = gpflow.kernels.SquaredExponential(lengthscales = [1.])# + gpflow.kernels.White(variance=0.001) # set up kernel
    data = (tf.convert_to_tensor(xfp2[cl_fp==i,:]), tf.convert_to_tensor(yfp2[cl_fp==i].flatten()[:,None]))
    m = gpflow.models.GPR(data=data, kernel=k, mean_function=gpflow.mean_functions.Constant(yfp2[cl_fp==i].flatten().mean())) # set up GPR model
    
    m.likelihood.variance.assign(0.005)
    p = m.likelihood.variance
    m.likelihood.variance = gpflow.Parameter(p, transform=tfp.bijectors.Sigmoid(f64(0.001), f64(0.01)) )    
    
    opt = gpflow.optimizers.Scipy() # set up hyperparameter optimization
    opt_logs = opt.minimize(m.training_loss, m.trainable_variables, method = 'tnc', options=dict(maxiter=100))  # run optimization
    temp, _ = m.predict_f(tf.convert_to_tensor(Xpred[cl_full==i,:])) # compute the mean and variance for the other samples in the phase region
    cameo_gpr_1b[cl_full==i] = temp.numpy().flatten()

r2_2a_cameo = r2_score(y_fp,cameo_gpr_1b)
acc_2a_cameo = f1_score(y_st, cl_full)
print( r2_2a_cameo, acc_2a_cameo)